In [1]:
# =========================
# Core
# =========================
import os
import json
import shutil
import random
import pickle
from collections import defaultdict

# =========================
# Numeric & Visualization
# =========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# =========================
# Computer Vision
# =========================
import cv2

# =========================
# Feature Extraction
# =========================
from skimage.feature import hog, local_binary_pattern

# =========================
# Machine Learning
# =========================
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score

# =========================
# Utils
# =========================
import joblib


In [2]:
import os

BASE_DIR = "./data"

# =========================
# Input directories
# =========================
ANN_DIR   = os.path.join(BASE_DIR, "annotations")
TRAIN_DIR = os.path.join(BASE_DIR, "train2017")
VAL_DIR   = os.path.join(BASE_DIR, "val2017")

TRAIN_ANN_PATH = os.path.join(ANN_DIR, "instances_train2017.json")
VAL_ANN_PATH   = os.path.join(ANN_DIR, "instances_val2017.json")

# =========================
# Output directories
# =========================
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

TRAIN_SUBSET_DIR = os.path.join(OUTPUT_DIR, "train_subset")
VAL_SUBSET_DIR   = os.path.join(OUTPUT_DIR, "val_subset")


TRAIN_SUBSET_ANN = os.path.join(
    TRAIN_SUBSET_DIR, "instances_train2017_subset.json"
)
VAL_SUBSET_ANN = os.path.join(
    VAL_SUBSET_DIR, "instances_val2017_subset.json"
)

# =========================
# Create directories
# =========================


print("✅ Paths ready (train & val, no test split)")


✅ Paths ready (train & val, no test split)


In [3]:
import os
import requests

ANNOT_URL = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
ANNOT_ZIP = "./data/annotations_trainval2017.zip"

os.makedirs(os.path.dirname(ANNOT_ZIP), exist_ok=True)

def download_annotation(url, out_path, batch_size=1024):
    if os.path.exists(out_path) and os.path.getsize(out_path) > 10 * 1024 * 1024:
        print(f"✅ Annotation ZIP already exists: {out_path}")
        return

    print("⬇️ Downloading annotations...")
    r = requests.get(url, stream=True)
    r.raise_for_status()

    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=batch_size):
            if chunk:
                f.write(chunk)

    print("✅ Annotation ZIP downloaded.")

download_annotation(ANNOT_URL, ANNOT_ZIP)


✅ Annotation ZIP already exists: ./data/annotations_trainval2017.zip


In [4]:
import zipfile

if os.path.exists(ANN_DIR) and len(os.listdir(ANN_DIR)) > 0:
    print(f"✅ Annotation Folder already exists: {ANN_DIR}")
else:
    with zipfile.ZipFile(ANNOT_ZIP, "r") as zip_ref:
        zip_ref.extractall("./data")
    print("Annotation extracted.")


✅ Annotation Folder already exists: ./data\annotations


In [5]:
import os
import requests

TRAIN_URL = "http://images.cocodataset.org/zips/train2017.zip"
TRAIN_ZIP = "./data/train_images.zip"

os.makedirs(os.path.dirname(TRAIN_ZIP), exist_ok=True)

def download_train(url, out_path, chunk_size=1024):
    if os.path.exists(out_path) and os.path.getsize(out_path) > 10 * 1024 * 1024:
        print(f"✅ ZIP already exists, skipping download: {out_path}")
        return

    print(f"⬇️ Downloading: {os.path.basename(out_path)} (this may take time)")
    r = requests.get(url, stream=True)
    r.raise_for_status()

    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            if chunk:
                f.write(chunk)

    print(f"✅ Download completed: {out_path}")

download_train(TRAIN_URL, TRAIN_ZIP)


✅ ZIP already exists, skipping download: ./data/train_images.zip


In [6]:
VAL_URL   = "http://images.cocodataset.org/zips/val2017.zip"
VAL_ZIP = "./data/val_images.zip"

os.makedirs(os.path.dirname(VAL_ZIP), exist_ok=True)

def download_train(url, out_path, chunk_size=1024):
    if os.path.exists(out_path) and os.path.getsize(out_path) > 10 * 1024 * 1024:
        print(f"✅ ZIP already exists, skipping download: {out_path}")
        return

    print(f"⬇️ Downloading: {os.path.basename(out_path)} (this may take time)")
    r = requests.get(url, stream=True)
    r.raise_for_status()

    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            if chunk:
                f.write(chunk)

    print(f"✅ Download completed: {out_path}")

download_train(TRAIN_URL, VAL_ZIP)


✅ ZIP already exists, skipping download: ./data/val_images.zip


In [7]:

if os.path.exists(TRAIN_DIR) and len(os.listdir(TRAIN_DIR)) > 0:
    print(f'Train Folder Already Created Skipping... : {TRAIN_DIR}')
else:
    with zipfile.ZipFile(TRAIN_ZIP, "r") as zip_ref:
        zip_ref.extractall("/content/data")
    print("Train images extracted.")


if os.path.exists(VAL_DIR) and len(os.listdir(VAL_DIR)) > 0:
    print(f'Val Folder Already Created Skipping...: {VAL_DIR}')
else:
    with zipfile.ZipFile(VAL_ZIP, "r") as zip_ref:
        zip_ref.extractall("/content/data")
    print("Test Image extracted.")


Train Folder Already Created Skipping... : ./data\train2017
Val Folder Already Created Skipping...: ./data\val2017


In [8]:
import json
import random


SUBSET_SIZE = 10000
TARGET_CATS = {1, 2, 3, 18}   # person, bicycle, car, dog


def build_coco_subset(ann_path, out_ann_path, split_name="train"):
    print(f"\n📦 Building {split_name} subset (imbalanced)")

    # =========================
    # Load annotation
    # =========================
    with open(ann_path, "r") as f:
        coco = json.load(f)

    # =========================
    # Filter annotations by target categories
    # =========================
    filtered_ann = [
        ann for ann in coco["annotations"]
        if ann["category_id"] in TARGET_CATS
    ]

    print(f"Filtered annotations (target cats): {len(filtered_ann)}")

    # =========================
    # Global random sampling (NO balancing)
    # =========================
    random.shuffle(filtered_ann)
    subset_ann = filtered_ann[:SUBSET_SIZE]

    print(f"Selected annotations: {len(subset_ann)}")

    # =========================
    # Collect corresponding images
    # =========================
    img_ids = {ann["image_id"] for ann in subset_ann}

    subset_imgs = [
        img for img in coco["images"]
        if img["id"] in img_ids
    ]

    print(f"Unique images in subset: {len(subset_imgs)}")

    # =========================
    # Save subset annotation
    # =========================
    subset_coco = {
        "images": subset_imgs,
        "annotations": subset_ann,
        "categories": coco["categories"]
    }

    with open(out_ann_path, "w") as f:
        json.dump(subset_coco, f)

    print(f"✅ {split_name} imbalanced subset saved: {out_ann_path}")


In [9]:
# Train subset
build_coco_subset(
    ann_path=TRAIN_ANN_PATH,
    out_ann_path=TRAIN_SUBSET_ANN,
    split_name="train"
)

# Validation subset
build_coco_subset(
    ann_path=VAL_ANN_PATH,
    out_ann_path=VAL_SUBSET_ANN,
    split_name="val"
)



📦 Building train subset (imbalanced)
Filtered annotations (target cats): 318953
Selected annotations: 10000
Unique images in subset: 8819
✅ train imbalanced subset saved: ./data\output\train_subset\instances_train2017_subset.json

📦 Building val subset (imbalanced)
Filtered annotations (target cats): 13470
Selected annotations: 10000
Unique images in subset: 2668
✅ val imbalanced subset saved: ./data\output\val_subset\instances_val2017_subset.json


In [10]:
from collections import Counter

def analyze_objects_per_image(annotations, split_name="train"):
    if not annotations:
        print(f"⚠️ No annotations found for {split_name}")
        return

    image_counts = Counter(ann["image_id"] for ann in annotations)

    max_obj = max(image_counts.values())
    avg_obj = sum(image_counts.values()) / len(image_counts)

    print(f"\n📊 Object statistics ({split_name})")
    print(f"Max objects in one image : {max_obj}")
    print(f"Avg objects per image    : {avg_obj:.2f}")


import json

def analyze_subset_from_file(subset_ann_path, split_name="train"):
    with open(subset_ann_path, "r") as f:
        coco = json.load(f)

    analyze_objects_per_image(coco["annotations"], split_name)


analyze_subset_from_file(TRAIN_SUBSET_ANN, split_name="train")
analyze_subset_from_file(VAL_SUBSET_ANN, split_name="val")


📊 Object statistics (train)
Max objects in one image : 4
Avg objects per image    : 1.13

📊 Object statistics (val)
Max objects in one image : 22
Avg objects per image    : 3.75


In [11]:
import os
import json
import numpy as np

def coco_to_object_pairs(
    img_dir,
    ann_path,
    selected_cats=None,
    split_name="train"
):
    """
    Convert COCO annotations into object-centric samples.
    For each image, only the largest object is kept.
    """

    print(f"\n🔄 Building object-centric pairs ({split_name})")

    with open(ann_path, "r") as f:
        data = json.load(f)

    img_map = {img["id"]: img for img in data["images"]}
    grouped = {}

    # =========================
    # Group annotations by image
    # =========================
    for ann in data["annotations"]:
        label = ann["category_id"]

        if selected_cats and label not in selected_cats:
            continue

        img_id = ann["image_id"]
        if img_id not in img_map:
            continue

        img_path = os.path.join(img_dir, img_map[img_id]["file_name"])
        if not os.path.exists(img_path):
            continue

        grouped.setdefault(
            img_id,
            {"img": img_path, "boxes": [], "labels": []}
        )

        grouped[img_id]["boxes"].append(ann["bbox"])
        grouped[img_id]["labels"].append(label)

    # =========================
    # Select largest object per image
    # =========================
    pairs = []
    for v in grouped.values():
        areas = [w * h for (_, _, w, h) in v["boxes"]]
        idx = int(np.argmax(areas))

        pairs.append({
            "img": v["img"],
            "bbox": v["boxes"][idx],
            "label": v["labels"][idx]
        })

    print(f"✅ Total samples ({split_name}): {len(pairs)}")
    return pairs


In [12]:
SELECTED = {1, 2, 3, 18}  # person, bicycle, car, dog
TRAIN_IMG_DIR = "./data/train2017"
VAL_IMG_DIR   = "./data/val2017"

Train_pairs = coco_to_object_pairs(
    img_dir=TRAIN_IMG_DIR,
    ann_path=TRAIN_SUBSET_ANN,
    selected_cats=SELECTED,
    split_name="train"
)

Val_pairs = coco_to_object_pairs(
    img_dir=VAL_IMG_DIR,
    ann_path=VAL_SUBSET_ANN,
    selected_cats=SELECTED,
    split_name="val"
)



🔄 Building object-centric pairs (train)
✅ Total samples (train): 8819

🔄 Building object-centric pairs (val)
✅ Total samples (val): 2668


In [13]:
from collections import Counter

def analyze_class_distribution(pairs, split_name="train"):
    labels = [p["label"] for p in pairs]
    counter = Counter(labels)

    total = sum(counter.values())

    print(f"\n📊 Class distribution ({split_name})")
    for lbl, cnt in counter.most_common():
        pct = 100 * cnt / total
        print(f"Class {lbl:>2} : {cnt:>6} samples ({pct:5.2f}%)")

    print(f"Total samples: {total}")
    return counter

train_dist = analyze_class_distribution(Train_pairs, split_name="train")



📊 Class distribution (train)
Class  1 :   7305 samples (82.83%)
Class  3 :   1153 samples (13.07%)
Class  2 :    182 samples ( 2.06%)
Class 18 :    179 samples ( 2.03%)
Total samples: 8819


In [14]:
def augment(img, label):
    if label == 1:  # person
        return img

    if random.random() < 0.5:
        img = cv2.flip(img, 1)

    if random.random() < 0.4:
        h, w = img.shape[:2]
        angle = random.uniform(-10, 10)
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
        img = cv2.warpAffine(img, M, (w,h), borderMode=cv2.BORDER_REFLECT_101)

    return img


In [15]:
from skimage.feature import hog, local_binary_pattern
import numpy as np
import cv2

LBP_P = 8
LBP_R = 1
LBP_BINS = LBP_P + 2  # WAJIB TETAP (10)

def extract_feature(img_path, bbox, label):
    img = cv2.imread(img_path)

    # 1. Pastikan image kebaca
    if img is None:
        return None

    H, W = img.shape[:2]
    x, y, w, h = map(int, bbox)

    # 2. Clamp bbox ke boundary image
    x1 = max(0, x)
    y1 = max(0, y)
    x2 = min(W, x + max(1, w))
    y2 = min(H, y + max(1, h))

    # 3. Validasi bbox
    if x2 <= x1 or y2 <= y1:
        return None

    crop = img[y1:y2, x1:x2]

    # 4. Double safety
    if crop.size == 0:
        return None

    crop = cv2.resize(crop, (256, 256))

    # 5. Augment (TRAIN only)
    crop = augment(crop, label)

    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)

    hog_feat = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm="L2-Hys"
    )

    lbp = local_binary_pattern(gray, P=LBP_P, R=LBP_R, method="uniform")

    lbp_hist, _ = np.histogram(
        lbp.ravel(),
        bins=LBP_BINS,
        range=(0, LBP_BINS),
        density=True
    )

    return np.concatenate([hog_feat, lbp_hist])


In [16]:
def extract_feature_eval(img_path, bbox):
    img = cv2.imread(img_path)
    if img is None:
        return None

    H, W = img.shape[:2]
    x, y, w, h = map(int, bbox)

    x1 = max(0, x)
    y1 = max(0, y)
    x2 = min(W, x + max(1, w))
    y2 = min(H, y + max(1, h))

    if x2 <= x1 or y2 <= y1:
        return None

    crop = img[y1:y2, x1:x2]
    if crop.size == 0:
        return None

    crop = cv2.resize(crop, (256, 256))

    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)

    hog_feat = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm="L2-Hys"
    )

    lbp = local_binary_pattern(gray, 8, 1, method="uniform")
    hist, _ = np.histogram(
        lbp.ravel(),
        bins=LBP_BINS,
        range=(0, LBP_BINS),
        density=True
    )

    return np.concatenate([hog_feat, hist])


In [17]:
from collections import Counter

dist = Counter(p["label"] for p in Train_pairs)
print(dist)
dist2 = Counter(p["label"] for p in Val_pairs)
print(dist2)

Counter({1: 7305, 3: 1153, 2: 182, 18: 179})
Counter({1: 2229, 3: 293, 18: 91, 2: 55})


In [18]:
from tqdm import tqdm
import numpy as np

X_train, y_train = [], []

for p in tqdm(Train_pairs, desc="Extracting TRAIN features"):
    feat = extract_feature(
        img_path=p["img"],
        bbox=p["bbox"],
        label=p["label"]
    )

    if feat is None:
        continue

    X_train.append(feat)
    y_train.append(p["label"])

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("Final TRAIN samples:", len(X_train))


Extracting TRAIN features: 100%|██████████| 8819/8819 [11:31<00:00, 12.75it/s] 


Final TRAIN samples: 8819


In [19]:
dropped = 0
for p in Train_pairs:
    if extract_feature(p["img"], p["bbox"], p["label"]) is None:
        dropped += 1

print("Dropped samples due to invalid bbox:", dropped)


Dropped samples due to invalid bbox: 0


In [20]:
X_val, y_val = [], []

for p in tqdm(Val_pairs, desc="Extracting VAL features"):
    feat = extract_feature_eval(
        img_path=p["img"],
        bbox=p["bbox"]
    )
    X_val.append(feat)
    y_val.append(p["label"])

X_val = np.asarray(X_val)
y_val = np.asarray(y_val)


Extracting VAL features: 100%|██████████| 2668/2668 [02:30<00:00, 17.76it/s]


In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# =========================
# Feature Scaling
# =========================
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

# # =========================
# # PCA (fit on TRAIN only)
# # =========================
# pca = PCA(
#     n_components=256,
#     svd_solver="randomized",
#     random_state=42
# )


# X_train_pca = pca.fit_transform(X_train_scaled)
# X_val_pca   = pca.transform(X_val_scaled)

# print(f"✅ PCA output dimension: {X_train_pca.shape[1]}")


In [22]:
from sklearn.metrics import f1_score

best_C = None
best_f1 = 0

for C in [0.5, 1.0, 2.0]:
    clf_tmp = LinearSVC(C=C, class_weight="balanced", max_iter=5000)
    clf_tmp.fit(X_train_scaled, y_train)

    y_pred = clf_tmp.predict(X_val_scaled)
    f1 = f1_score(y_val, y_pred, average="macro")

    print(f"C={C} → Macro F1={f1:.3f}")

    if f1 > best_f1:
        best_f1 = f1
        best_C = C

print("Best C:", best_C)


C=0.5 → Macro F1=0.372
C=1.0 → Macro F1=0.372
C=2.0 → Macro F1=0.372
Best C: 0.5


In [23]:
clf = LinearSVC(C=best_C, class_weight="balanced", max_iter=5000)
clf.fit(X_train_scaled, y_train)

,"penalty penalty: {'l1', 'l2'}, default='l2'Specifies the norm used in the penalization. The 'l2'penalty is the standard used in SVC. The 'l1' leads to ``coef_``vectors that are sparse.",'l2'
,"loss loss: {'hinge', 'squared_hinge'}, default='squared_hinge'Specifies the loss function. 'hinge' is the standard SVM loss(used e.g. by the SVC class) while 'squared_hinge' is thesquare of the hinge loss. The combination of ``penalty='l1'``and ``loss='hinge'`` is not supported.",'squared_hinge'
,"dual dual: ""auto"" or bool, default=""auto""Select the algorithm to either solve the dual or primaloptimization problem. Prefer dual=False when n_samples > n_features.`dual=""auto""` will choose the value of the parameter automatically,based on the values of `n_samples`, `n_features`, `loss`, `multi_class`and `penalty`. If `n_samples` < `n_features` and optimizer supportschosen `loss`, `multi_class` and `penalty`, then dual will be set to True,otherwise it will be set to False... versionchanged:: 1.3 The `""auto""` option is added in version 1.3 and will be the default in version 1.5.",'auto'
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive.For an intuitive visualization of the effects of scalingthe regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",0.5
,"multi_class multi_class: {'ovr', 'crammer_singer'}, default='ovr'Determines the multi-class strategy if `y` contains more thantwo classes.``""ovr""`` trains n_classes one-vs-rest classifiers, while``""crammer_singer""`` optimizes a joint objective over all classes.While `crammer_singer` is interesting from a theoretical perspectiveas it is consistent, it is seldom used in practice as it rarely leadsto better accuracy and is more expensive to compute.If ``""crammer_singer""`` is chosen, the options loss, penalty and dualwill be ignored.",'ovr'
,"fit_intercept fit_intercept: bool, default=TrueWhether or not to fit an intercept. If set to True, the feature vectoris extended to include an intercept term: `[x_1, ..., x_n, 1]`, where1 corresponds to the intercept. If set to False, no intercept will beused in calculations (i.e. data is expected to be already centered).",True
,"intercept_scaling intercept_scaling: float, default=1.0When `fit_intercept` is True, the instance vector x becomes ``[x_1,..., x_n, intercept_scaling]``, i.e. a ""synthetic"" feature with aconstant value equal to `intercept_scaling` is appended to the instancevector. The intercept becomes intercept_scaling * synthetic featureweight. Note that liblinear internally penalizes the intercept,treating it like any other term in the feature vector. To reduce theimpact of the regularization on the intercept, the `intercept_scaling`parameter can be set to a value greater than 1; the higher the value of`intercept_scaling`, the lower the impact of regularization on it.Then, the weights become `[w_x_1, ..., w_x_n,w_intercept*intercept_scaling]`, where `w_x_1, ..., w_x_n` representthe feature weights and the intercept weight is scaled by`intercept_scaling`. This scaling allows the intercept term to have adifferent regularization behavior compared to the other features.",1
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to ``class_weight[i]*C`` forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",'balanced'
,"verbose verbose: int, default=0Enable verbose output. Note that this setting takes advantage of aper-process runtime setting in liblinear that, if enabled, may not workproperly in a multithreaded context.",0
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseud

In [24]:
y_val_pred = clf.predict(X_val_scaled)

label_map = {1:"person", 2:"bicycle", 3:"car", 18:"dog"}

print(classification_report(
    y_val, y_val_pred,
    target_names=[label_map[c] for c in sorted(label_map)]
))


              precision    recall  f1-score   support

      person       0.88      0.89      0.89      2229
     bicycle       0.29      0.09      0.14        55
         car       0.38      0.48      0.43       293
         dog       0.10      0.02      0.04        91

    accuracy                           0.80      2668
   macro avg       0.41      0.37      0.37      2668
weighted avg       0.79      0.80      0.79      2668



In [25]:
import joblib
import os

OUTPUT_DIR = './NON-PCA'

SAVE_DIR = os.path.join(OUTPUT_DIR, "models")
os.makedirs(SAVE_DIR, exist_ok=True)

joblib.dump(clf,    os.path.join(SAVE_DIR, "svm_hoglbp_ver1.pkl"))
joblib.dump(scaler, os.path.join(SAVE_DIR, "scaler_ver1.pkl"))

print("✅ Model artifacts saved")

✅ Model artifacts saved


Interferencd With YOLO

In [26]:
from ultralytics import YOLO
yolo = YOLO("yolov8n.pt")

def predict_test_image(img_path):
    res = yolo(img_path, conf=0.4, verbose=False)
    if len(res[0].boxes) == 0:
        return "No object"

    boxes = res[0].boxes.xyxy.cpu().numpy()
    areas = (boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1])
    i = areas.argmax()

    x1,y1,x2,y2 = boxes[i]
    bbox = (int(x1), int(y1), int(x2-x1), int(y2-y1))

    feat = extract_feature_eval(img_path, bbox)
    feat = scaler.transform(feat.reshape(1,-1))

    pred = clf.predict(feat)[0]
    return label_map[pred]


In [27]:
img_name = './testimage.png'
print(img_name, "→", predict_test_image('./testimage.png'))


./testimage.png → bicycle
